# Merger Tree


In [1]:
import os
import pandas as pd
import numpy as np

In [6]:
filename = '../Preprocess/halo-csv/GadgetX-NewMDCLUSTER_0001.snap_022.z9.454.csv'
os.path.exists(filename)

True

In [7]:
df = pd.read_csv(filename)

In [8]:
df.head()

,ID,hostHalo,numSubStruct,Mvir,npart,Xc,Yc,Zc,VXc,VYc,...,Ecy,Ecz,ovdens,nbins,fMhires,Ekin,Epot,SurfP,Phi0,cNFW
0,22000000000001,0,0,1.354260e+11,172,513239.886080,509424.984455,504994.503089,-241.78,-250.71,...,0.531913,-0.076600,198.68,10,1.0,3.869010e+15,-7.310530e+15,-1.294300e+15,261245.0,7.48688
1,22000000000002,0,0,9.675780e+10,121,503443.525897,482758.730650,509472.744332,-78.24,176.05,...,0.236329,0.662625,171.72,9,1.0,2.234180e+15,-4.083920e+15,-6.776860e+14,213894.0,7.78758
2,22000000000003,0,0,9.764200e+10,116,497940.989004,517144.951555,497178.955211,-62.07,-257.87,...,0.935221,-0.343743,195.06,9,1.0,2.569460e+15,-4.403800e+15,-4.118720e+14,153270.0,10.32600
3,22000000000004,0,0,8.306050e+10,98,501419.999098,485557.123234,481604.303184,59.89,-49.27,...,0.891828,-0.438457,194.95,8,1.0,2.083690e+15,-3.352930e+15,-4.293580e+14,140979.0,11.05420
4,22000000000005,0,0,7.591840e+10,94,478707.576620,497921.055761,491840.417015,76.73,-22.95,...,0.596216,-0.647032,199.72,8,1.0,1.409270e+15,-2.744780e+15,-4.158670e+14,133767.0,8.10470


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 43 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            126 non-null    int64  
 1   hostHalo      126 non-null    int64  
 2   numSubStruct  126 non-null    int64  
 3   Mvir          126 non-null    float64
 4   npart         126 non-null    int64  
 5   Xc            126 non-null    float64
 6   Yc            126 non-null    float64
 7   Zc            126 non-null    float64
 8   VXc           126 non-null    float64
 9   VYc           126 non-null    float64
 10  VZc           126 non-null    float64
 11  Rvir          126 non-null    float64
 12  Rmax          126 non-null    float64
 13  r2            126 non-null    float64
 14  mbp_offset    126 non-null    float64
 15  com_offset    126 non-null    float64
 16  Vmax          126 non-null    float64
 17  v_esc         126 non-null    float64
 18  sigV          126 non-null    

In [11]:
# Calculate features for halo pairs
def calculate_features_for_pairs(pairs, df, merge_status):
    mass1 = []
    mass2 = []
    relative_positions = []
    relative_velocities = []
    will_merge = []
    numSubStruct1 = []
    numSubStruct2 = []
    sigV1 = []
    sigV2 = []
    lambda1 = []
    lambda2 = []
    cNFW1 = []
    cNFW2 = []
    Ekin1 = []
    Ekin2 = []
    Epot1 = []
    Epot2 = []
    ovdens1 = []
    ovdens2 = []

    for pair in pairs:
        halo1 = df.loc[pair[0]]
        halo2 = df.loc[pair[1]]

        # Calculate the relative position and velocity
        position1 = np.array([halo1['Xc'], halo1['Yc'], halo1['Zc']])
        position2 = np.array([halo2['Xc'], halo2['Yc'], halo2['Zc']])
        velocity1 = np.array([halo1['VXc'], halo1['VYc'], halo1['VZc']])
        velocity2 = np.array([halo2['VXc'], halo2['VYc'], halo2['VZc']])

        relative_position = np.linalg.norm(position1 - position2)
        relative_velocity = np.linalg.norm(velocity1 - velocity2)

        # Store the features
        mass1.append(halo1['Mvir'])
        mass2.append(halo2['Mvir'])
        relative_positions.append(relative_position)
        relative_velocities.append(relative_velocity)
        will_merge.append(merge_status)
        numSubStruct1.append(halo1['numSubStruct'])
        numSubStruct2.append(halo2['numSubStruct'])
        sigV1.append(halo1['sigV'])
        sigV2.append(halo2['sigV'])
        lambda1.append(halo1['lambda'])
        lambda2.append(halo2['lambda'])
        cNFW1.append(halo1['cNFW'])
        cNFW2.append(halo2['cNFW'])
        Ekin1.append(halo1['Ekin'])
        Ekin2.append(halo2['Ekin'])
        Epot1.append(halo1['Epot'])
        Epot2.append(halo2['Epot'])
        ovdens1.append(halo1['ovdens'])
        ovdens2.append(halo2['ovdens'])
        
    return mass1, mass2, relative_positions, relative_velocities, will_merge, numSubStruct1, numSubStruct2, sigV1, sigV2, lambda1, lambda2, cNFW1, cNFW2, Ekin1, Ekin2, Epot1, Epot2, ovdens1, ovdens2

In [23]:
pair = df.loc[[0,1],:]
pair

,ID,hostHalo,numSubStruct,Mvir,npart,Xc,Yc,Zc,VXc,VYc,...,Ecy,Ecz,ovdens,nbins,fMhires,Ekin,Epot,SurfP,Phi0,cNFW
0,22000000000001,0,0,1.354260e+11,172,513239.886080,509424.984455,504994.503089,-241.78,-250.71,...,0.531913,-0.076600,198.68,10,1.0,3.869010e+15,-7.310530e+15,-1.294300e+15,261245.0,7.48688
1,22000000000002,0,0,9.675780e+10,121,503443.525897,482758.730650,509472.744332,-78.24,176.05,...,0.236329,0.662625,171.72,9,1.0,2.234180e+15,-4.083920e+15,-6.776860e+14,213894.0,7.78758


In [ ]:
# Defining the model

model = Sequential()
model.add(Dense(18,input_shape=(18,), activation='relu'))
#model.add(Normalization())
#model.add(Dropout(0.5))
model.add(Dense(12, activation='relu'))
#model.add(Normalization())
#model.add(Dropout(0.5))
model.add(Dense(6, activation='relu'))
#model.add(Normalization())
#model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification